<a href="https://colab.research.google.com/github/alanpirotta/freecodecamp_certif/blob/main/feature_engineering_health_costs_with_regression_alternative_way.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

### Change categorical data to numbers.
Binary if possible

In [ ]:
# feature = "region"
# classes = dataset[feature].unique().tolist()
# print(f"Feature classes: {classes}")

# dataset[feature] = dataset[feature].map(classes.index)

# More manual Alternative 
# dataset['region'] = dataset['region'].replace({'southeast': 0,
#                                               'southwest': 1,
#                                               'northwest': 2,
#                                               'northeast': 3
#                                             })
# print(dataset['region'].value_counts())
# dataset.tail()

In [ ]:
# dataset['smoker'] = np.where(dataset['smoker']=='yes',1,0)
# print(dataset['smoker'].value_counts())
# dataset.tail()

In [ ]:
# dataset['sex'] = np.where(dataset['sex']=='male',1,0)
# print(dataset['sex'].value_counts())
# dataset.tail()

### Split between train and test datasets

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(dataset, test_size=0.2)
print(f'train shape: {train_df.shape}')
print(f'test shape: {test_df.shape}')
train_df.tail()

In [ ]:
train_labels = train_df.pop('expenses')
test_labels = test_df.pop('expenses')

### Feature engineering

####region OneHotEncoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

In [ ]:
encoder = OneHotEncoder() 
encoder.fit(train_df[['region']])
re = encoder.transform(train_df[['region']]).todense()
type(re)

In [ ]:
re_df = pd.DataFrame(re, columns=encoder.categories_, index=train_df[['region']].index)
re_df

Last cells where to test OneHotEncoder, next cells i'll make the transform to all applicable features.

In [ ]:
one_hot_encode = ColumnTransformer ([
                                     (
                                      'one_hot_encode',   #Transformation name
                                      OneHotEncoder(sparse=False),   #transformer to use
                                      ['region', 'sex', 'smoker']    #features to transform
                                     )
                  ])
one_hot_encode.fit(train_df)

In [ ]:
region_sex_smoker = one_hot_encode.transform(train_df)
region_sex_smoker_df = pd.DataFrame(region_sex_smoker, index=train_df.index)

In [ ]:
region_sex_smoker_test = one_hot_encode.transform(test_df)
region_sex_smoker_test_df = pd.DataFrame(region_sex_smoker_test, index=test_df.index)

####Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler

In [ ]:
# scaler = MaxAbsScaler()
# scaler.fit(train_df[['bmi']])
# bmi_scaled = scaler.transform(train_df[['bmi']])

In [ ]:
# pd.DataFrame({'original': train_df['bmi'].values, 'scaled': bmi_scaled.squeeze()}).describe()

In [ ]:
# scaled_bmi_df = pd.DataFrame(bmi_scaled, columns=['bmi_scaled'], index=train_df[['bmi']].index)

In [ ]:
scaler_encoding = ColumnTransformer([
                            (
                             'MinMaxScaler_encoding',
                             MinMaxScaler(),
                             ['age', 'bmi']
                            )
          ])
scaler_encoding.fit(train_df)

In [ ]:
age_bmi = scaler_encoding.transform(train_df)
age_bmi_df = pd.DataFrame(age_bmi, index=train_df.index)

In [ ]:
age_bmi_test = scaler_encoding.transform(test_df)
age_bmi_test_df = pd.DataFrame(age_bmi_test, index=test_df.index)

In [ ]:
r_scaler_encoding = ColumnTransformer([
                            (
                             'RobustScaler_encoding',
                             RobustScaler(),
                             ['children']
                            )
          ])
r_scaler_encoding.fit(train_df)

In [ ]:
children = r_scaler_encoding.transform(train_df)
children_df = pd.DataFrame(children, index=train_df.index)

In [ ]:
children_test = r_scaler_encoding.transform(test_df)
children_test_df = pd.DataFrame(children_test, index=test_df.index)

In [ ]:
train_dataset = pd.concat([region_sex_smoker_df, age_bmi_df,children_df], axis=1)
train_dataset.shape

In [ ]:
test_dataset = pd.concat([region_sex_smoker_test_df, age_bmi_test_df,children_test_df], axis=1)
test_dataset.shape

###Quick Test with linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
lr = LinearRegression()

In [ ]:
lr.fit(train_dataset, train_labels)
train_prediction = lr.predict(train_dataset)
train_mae = mean_absolute_error(train_prediction, train_labels)
train_mae

In [ ]:
test_pred = lr.predict(test_dataset)
test_mae = mean_absolute_error(test_pred, test_labels)
test_mae

In [ ]:
print(f'Train score: {lr.score(train_dataset,train_labels)}')
print(f'Test score: {lr.score(test_dataset,test_labels)}')

###Model

In [ ]:
normalizer = layers.experimental.preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))

In [ ]:
model = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(2, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=['mae', 'mse'])

In [ ]:
history = model.fit(
    train_dataset, train_labels, 
    epochs=100,
    # suppress logging
    # verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print('loss',loss,'mae',mae,'mse',mse)

###Partial-Conclusion:
With all the feature-engineering i did, the linear regression score stayed practically the same, but the MAE decreased 13% from the results with only labelEncoding.

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


Conclusion: This model works great for lower expenses and passable for really high ones, but in the middle doesn't work. The original way i tried with randomForest achieved a better result. I think this model can be improved.